In [1]:
import trino
import pandas as pd
import numpy as np
import logging
from datetime import datetime

# Configure logging
logging.basicConfig(level=logging.INFO,
                    format='%(asctime)s - %(levelname)s - %(message)s',
                    datefmt='%Y-%m-%d %H:%M:%S')
logger = logging.getLogger(__name__)

### AC Fault Code:
1. No_Fault
2. Low_Voltage
3. Outside_Temp_Sensor_Fault
4. High_Voltage
5. Exhaust_Temp_Protection
6. Eva_Temp_Sesnor_Fault
7. AC Communication Fail

### AC Status:
1. Start
2. Stop

### TMS Fault Code:
1. No Fault
2. Water_Sensor_Failure
3. Water_Pump_Failure
4. Water_IN_Sensor_Failure
5. Exhaust_Temp_Protection
6. Low_Water_Level_Alarm
7. LV Undervoltage

### TMS Working Mode:
1. Charging_Cooling
2. Fast_Discharge_Cooling
3. Self_Circulation
4. Low_Coolant
5. Off

### B2T TMS Control Cmd:
1. Charging_Cooling
2. Fast_Discharge_Cooling
3. Self_Circulation
4. Off

In [2]:
def get_all_battery_data(start_date, end_date, vehicle_ids):
    """
    Fetch all battery data for the specified date range and vehicle IDs in one go.
    
    Args:
        start_date: Start date in 'YYYY-MM-DD' format
        end_date: End date in 'YYYY-MM-DD' format
        vehicle_ids: List of vehicle IDs
        
    Returns:
        DataFrame with battery data
    """
    logger.info(f"Fetching all battery data from {start_date} to {end_date} for vehicles {vehicle_ids}")
    
    # Format vehicle IDs for the query
    vehicle_ids_str = ', '.join([f"'{vid}'" for vid in vehicle_ids])
    
    # Connect to Trino
    conn = trino.dbapi.connect(
        host="trino",
        port=8080,
        user="admin",
        catalog="adhoc",
        schema="default"
    )

    # Query for cpo100 data
    cpo100_query = f"""
    SELECT 
        id, CAST(timestamp AS TIMESTAMP) AT TIME ZONE 'Asia/Kolkata' as timestamp, dt, 
        date_trunc('minute', CAST(timestamp AS TIMESTAMP) AT TIME ZONE 'Asia/Kolkata') as ts_mins,
        batterycoolingstate, batterycoolanttemperature,
        temperaturedifferencealarm, chargingcurrentalarm, dischargecurrentalarm,
        vehiclereadycondition, gun_connection_status, ignitionstatus,
        vehicle_speed_vcu,gear_position,
        pack1_cellmax_temperature, pack1_maxtemperature_cell_number,
        pack1_cell_min_temperature, pack1_celltemperature_cellnumber
    FROM 
        facts_prod.can_parsed_output_100
    WHERE 
        id IN ({vehicle_ids_str})
        AND DATE(timestamp) >= DATE('{start_date}')
        AND DATE(timestamp) <= DATE('{end_date}')
    """

    # Query for can_ac data
    can_ac_query = f"""
    SELECT 
        id, CAST(timestamp AS TIMESTAMP) AT TIME ZONE 'Asia/Kolkata' as timestamp, date,
        date_trunc('minute', CAST(timestamp AS TIMESTAMP) AT TIME ZONE 'Asia/Kolkata') as ts_mins_cac,    
        b2t_tms_control_cmd, b2t_set_water_out_temp, b2t_battery_min_temp, b2t_battery_max_temp,
        tms_working_mode, tms_fault_code, coolant_out_temp, coolant_in_temp,
        (coolant_out_temp - coolant_in_temp) as coolant_temp_diff,
        comp_status,comp_target_hz as comp_target_freq,
        comp_running_frequency as comp_running_freq,
        v2t_vehicle_coolant_low, comp_current
    FROM 
        facts_prod.can_output_ac
    WHERE
        id IN ({vehicle_ids_str})
        AND DATE(timestamp) >= DATE('{start_date}')
        AND DATE(timestamp) <= DATE('{end_date}')
        and b2t_battery_min_temp > 0
        and b2t_battery_max_temp > 0
        and coolant_in_temp > 0
    """

    # Execute queries and fetch data
    cur = conn.cursor()

    # Fetch cpo100 data
    cur.execute(cpo100_query)
    cpo100_columns = [desc[0] for desc in cur.description]
    cpo100_rows = cur.fetchall()
    df_cpo100 = pd.DataFrame(cpo100_rows, columns=cpo100_columns)

    # Fetch can_ac data
    cur.execute(can_ac_query)
    can_ac_columns = [desc[0] for desc in cur.description]
    can_ac_rows = cur.fetchall()
    df_can_ac = pd.DataFrame(can_ac_rows, columns=can_ac_columns)

    # Close connections
    cur.close()
    conn.close()

    # Convert timestamp columns to datetime if they aren't already
    df_cpo100['timestamp'] = pd.to_datetime(df_cpo100['timestamp'])
    df_cpo100['ts_mins'] = pd.to_datetime(df_cpo100['ts_mins'])
    df_can_ac['timestamp'] = pd.to_datetime(df_can_ac['timestamp'])
    df_can_ac['ts_mins_cac'] = pd.to_datetime(df_can_ac['ts_mins_cac'])

    # Add row numbers (equivalent to SQL row_number() window function)
    # For cpo100
    df_cpo100 = df_cpo100.sort_values(['id', 'ts_mins', 'timestamp'])
    df_cpo100['cpo_rn'] = df_cpo100.groupby(['id', 'ts_mins']).cumcount() + 1

    # For can_ac
    df_can_ac = df_can_ac.sort_values(['id', 'ts_mins_cac', 'timestamp'])
    df_can_ac['cac_rn'] = df_can_ac.groupby(['id', 'ts_mins_cac']).cumcount() + 1

    # Perform the join in pandas (equivalent to SQL right join)
    merged_df = pd.merge(
        df_can_ac,
        df_cpo100,
        left_on=['id', 'ts_mins_cac', 'cac_rn'],
        right_on=['id', 'ts_mins', 'cpo_rn'],
        how='left',
        suffixes=('_can_ac','_cpo100')
    )

    # Select only the columns we need
    result_columns = []

    # Add id column
    if 'id' in merged_df.columns:
        result_columns.append('id')
    elif 'id_cpo100' in merged_df.columns:
        result_columns.append('id_cpo100')

    # Add other columns from cpo100
    cpo100_cols = ['dt', 'ignitionstatus', 'vehiclereadycondition', 'gun_connection_status','vehicle_speed_vcu',
                   'gear_position','pack1_cellmax_temperature', 'pack1_cell_min_temperature']

    for col in cpo100_cols:
        if col in merged_df.columns:
            result_columns.append(col)
        elif f'{col}_cpo100' in merged_df.columns:
            result_columns.append(f'{col}_cpo100')

    # Add timestamp (from cpo100)
    if 'timestamp_cpo100' in merged_df.columns:
        result_columns.append('timestamp_cpo100')
    elif 'timestamp' in merged_df.columns:
        result_columns.append('timestamp')

    # Add ts_mins (from cpo100)
    if 'ts_mins_cpo100' in merged_df.columns:
        result_columns.append('ts_mins_cpo100')
    elif 'ts_mins' in merged_df.columns:
        result_columns.append('ts_mins')

    # Add columns from can_ac
    can_ac_cols = ['b2t_tms_control_cmd', 'b2t_set_water_out_temp', 
                  'b2t_battery_min_temp', 'b2t_battery_max_temp', 'tms_working_mode',
                  'coolant_out_temp', 'coolant_in_temp', 'coolant_temp_diff', 'comp_status','tms_fault_code']

    for col in can_ac_cols:
        if col in merged_df.columns:
            result_columns.append(col)
        elif f'{col}_can_ac' in merged_df.columns:
            result_columns.append(f'{col}_can_ac')

    # Create result DataFrame with selected columns
    result_df = merged_df[result_columns].copy()

    # Rename columns to match original query output
    rename_dict = {}
    if 'id_cpo100' in result_df.columns:
        rename_dict['id_cpo100'] = 'id'
    if 'timestamp_cpo100' in result_df.columns:
        rename_dict['timestamp_cpo100'] = 'timestamp'
    if 'ts_mins_cpo100' in result_df.columns:
        rename_dict['ts_mins_cpo100'] = 'ts_mins'

    # Rename any columns with _can_ac suffix
    for col in result_df.columns:
        if col.endswith('_can_ac'):
            rename_dict[col] = col[:-7]  # Remove the '_can_ac' suffix

    result_df = result_df.rename(columns=rename_dict)

    # Sort the result as in the original query
    result_df = result_df.sort_values(['id', 'timestamp'])

    logger.info(f"Retrieved {len(result_df)} battery data records")
    return result_df

In [3]:
def preprocess_data(df):
    """Clean and preprocess the data, including imputation for gun_connection_status."""
    # Create a copy for cleaning
    clean_df = df.copy()

    # Ensure timestamp columns are datetime
    timestamp_cols = ['timestamp', 'ts_mins']
    for col in timestamp_cols:
        if col in clean_df.columns:
            clean_df[col] = pd.to_datetime(clean_df[col], errors='coerce')

    # Convert numeric columns
    numeric_cols = ['id', 'ignitionstatus', 'vehiclereadycondition', 
                   'gun_connection_status', 'vehicle_speed_vcu', 'gear_position',
                   'pack1_cellmax_temperature', 'pack1_cell_min_temperature',
                   'b2t_set_water_out_temp', 'b2t_battery_min_temp', 
                   'b2t_battery_max_temp', 'coolant_out_temp', 'coolant_in_temp',
                   'coolant_temp_diff']

    for col in numeric_cols:
        if col in clean_df.columns:
            clean_df[col] = pd.to_numeric(clean_df[col], errors='coerce')

    # Handle outliers (-40°C)
    temp_cols = ['pack1_cellmax_temperature', 'pack1_cell_min_temperature',
               'b2t_battery_min_temp', 'b2t_battery_max_temp',
               'coolant_out_temp', 'coolant_in_temp']

    for col in temp_cols:
        if col in clean_df.columns:
            # Replace -40 with NaN (outliers)
            clean_df.loc[clean_df[col] == -40, col] = np.nan

    # Impute missing gun_connection_status values
    # First, forward fill within each vehicle's data
    clean_df['gun_connection_status'] = clean_df.groupby('id')['gun_connection_status'].ffill()
    
    # If there are still missing values (at the beginning), use 0 (not connected)
    clean_df['gun_connection_status'] = clean_df['gun_connection_status'].fillna(0).astype(int)
    
    # Convert to integer (0 or 1)
    clean_df['gun_connection_status'] = clean_df['gun_connection_status'].astype(int)

    # Remove rows with critical missing values
    critical_cols = ['b2t_tms_control_cmd', 'tms_working_mode', 
                   'b2t_battery_max_temp', 'b2t_battery_min_temp']
    original_count = len(clean_df)
    clean_df.dropna(subset=critical_cols, inplace=True)

    logger.info(f"Data after cleaning: {clean_df.shape} (removed {original_count - len(clean_df)} rows)")
    
    # Log distribution of gun_connection_status
    gun_status_counts = clean_df['gun_connection_status'].value_counts()
    logger.info(f"Gun connection status distribution: {gun_status_counts.to_dict()}")

    return clean_df

In [4]:
def identify_charging_events(clean_df):
    """
    Identify charging events based on gun_connection_status = 1.
    
    Args:
        clean_df: Preprocessed dataframe with battery data
        
    Returns:
        DataFrame with charging event information
    """
    logger.info("Identifying charging events based on gun_connection_status")
    
    # Create a copy to avoid modifying the original
    df = clean_df.copy()
    
    # Sort by vehicle and timestamp
    df = df.sort_values(['id', 'timestamp'])
    
    # Create a column indicating when gun_connection_status changes
    df['gun_status_change'] = (df['gun_connection_status'] != df.groupby('id')['gun_connection_status'].shift(1)).astype(int)
    
    # Create a column indicating the start of a charging event (gun_connection_status becomes 1)
    df['charging_start'] = ((df['gun_connection_status'] == 1) & 
                           (df.groupby('id')['gun_connection_status'].shift(1) == 0)).astype(int)
    
    # Create a column indicating the end of a charging event (gun_connection_status becomes 0)
    df['charging_end'] = ((df['gun_connection_status'] == 0) & 
                         (df.groupby('id')['gun_connection_status'].shift(1) == 1)).astype(int)
    
    # Get all rows where charging starts or ends
    transition_points = df[(df['charging_start'] == 1) | (df['charging_end'] == 1)].copy()
    
    # Initialize list to store charging events
    charging_events = []
    
    # Process each vehicle separately
    for vehicle_id in df['id'].unique():
        vehicle_data = df[df['id'] == vehicle_id].copy()
        vehicle_transitions = transition_points[transition_points['id'] == vehicle_id].copy()
        
        # Sort by timestamp
        vehicle_transitions = vehicle_transitions.sort_values('timestamp')
        
        # Initialize variables
        current_start = None
        current_end = None
        
        # Process each transition point
        for idx, row in vehicle_transitions.iterrows():
            if row['charging_start'] == 1:
                # Start of a charging event
                current_start = row['timestamp']
            elif row['charging_end'] == 1 and current_start is not None:
                # End of a charging event
                current_end = row['timestamp']
                
                # Create event record
                event = {
                    'vehicle_id': vehicle_id,
                    'start_time': current_start,
                    'end_time': current_end,
                    'duration_seconds': (current_end - current_start).total_seconds()
                }
                
                # Get battery data for this event
                event_data = vehicle_data[(vehicle_data['timestamp'] >= current_start) & 
                                        (vehicle_data['timestamp'] <= current_end)].copy()
                
                # Add additional event information
                if len(event_data) > 0:
                    event['data_points'] = len(event_data)
                    event['min_battery_temp'] = event_data['b2t_battery_max_temp'].min()
                    event['max_battery_temp'] = event_data['b2t_battery_max_temp'].max()
                    event['avg_battery_temp'] = event_data['b2t_battery_max_temp'].mean()
                    
                    # Add event to list
                    charging_events.append(event)
                
                # Reset for next event
                current_start = None
    
    # Convert to DataFrame
    if charging_events:
        events_df = pd.DataFrame(charging_events)
        logger.info(f"Identified {len(events_df)} charging events")
        return events_df
    else:
        logger.info("No charging events identified")
        return pd.DataFrame()

In [5]:
def assign_events_to_battery_data(battery_df, events_df):
    """
    Assign each battery data record to its corresponding charging event.
    
    Args:
        battery_df: DataFrame with battery data
        events_df: DataFrame with charging events
        
    Returns:
        DataFrame with battery data and event information
    """
    logger.info("Assigning battery data to charging events")
    
    # Create a copy of the battery dataframe
    result_df = battery_df.copy()
    
    # Initialize columns for event information
    result_df['event_id'] = -1
    result_df['event_start_time'] = pd.NaT
    result_df['event_end_time'] = pd.NaT
    
    # For each vehicle, assign events
    for vehicle_id in battery_df['id'].unique():
        # Get battery data for this vehicle
        vehicle_battery = battery_df[battery_df['id'] == vehicle_id].copy()
        
        # Get events for this vehicle
        vehicle_events = events_df[events_df['vehicle_id'] == vehicle_id].copy()
        
        # Skip if no events for this vehicle
        if vehicle_events.empty:
            continue
        
        # Sort events by start time
        vehicle_events = vehicle_events.sort_values('start_time')
        
        # For each event, find battery data within the event time range
        for event_idx, event in vehicle_events.iterrows():
            # Create mask for battery data within this event
            mask = (vehicle_battery['timestamp'] >= event['start_time']) & \
                   (vehicle_battery['timestamp'] <= event['end_time'])
            
            # Assign event information to battery data
            result_df.loc[mask & (result_df['id'] == vehicle_id), 'event_id'] = event_idx
            result_df.loc[mask & (result_df['id'] == vehicle_id), 'event_start_time'] = pd.to_datetime(event['start_time'])
            result_df.loc[mask & (result_df['id'] == vehicle_id), 'event_end_time'] = pd.to_datetime(event['end_time'])
    
    # Filter out battery data that doesn't belong to any event
    result_df = result_df[result_df['event_id'] != -1].copy()
    
    logger.info(f"Assigned {len(result_df)} battery data records to events")
    return result_df

In [6]:
def get_transition_data(clean_df):
    """
    Create a dataframe showing transitions in b2t_tms_control_cmd and tms_working_mode.
    
    Args:
        clean_df: Preprocessed dataframe with battery data
        
    Returns:
        DataFrame with transition information including timestamps and multiple temperature readings
    """
    logger.info("Identifying transitions in BCS system")
    
    # Sort by vehicle and timestamp to ensure proper ordering
    df = clean_df.sort_values(['id', 'timestamp']).copy()
    
    # Create a mask for the first row of each vehicle
    df['is_first'] = df.groupby('id').cumcount() == 0
    
    # Get previous state for both control commands
    df['prev_cmd'] = df.groupby('id')['b2t_tms_control_cmd'].shift(1)
    df['prev_mode'] = df.groupby('id')['tms_working_mode'].shift(1)
    
    # For the first row, set previous state to 'Unknown'
    df.loc[df['is_first'], 'prev_cmd'] = 'Unknown'
    df.loc[df['is_first'], 'prev_mode'] = 'Unknown'
    
    # Create masks for transitions (including first row)
    df['cmd_changed'] = (df['b2t_tms_control_cmd'] != df['prev_cmd']) | df['is_first']
    df['mode_changed'] = (df['tms_working_mode'] != df['prev_mode']) | df['is_first']
    
    # We want rows where either command changed
    mask = df['cmd_changed'] | df['mode_changed']
    transition_df = df[mask].copy()
    
    # Select and rename columns of interest
    transition_df = transition_df[[
        'id', 'timestamp', 'prev_cmd', 'b2t_tms_control_cmd', 
        'prev_mode', 'tms_working_mode', 
        'b2t_battery_min_temp', 'b2t_battery_max_temp',
        'coolant_in_temp', 'coolant_out_temp',
        'ignitionstatus', 'vehiclereadycondition', 'gun_connection_status',
        'event_id', 'event_start_time', 'event_end_time',
        'cmd_changed', 'mode_changed'
    ]]
    
    # Rename columns for clarity
    transition_df = transition_df.rename(columns={
        'b2t_tms_control_cmd': 'current_cmd',
        'tms_working_mode': 'current_mode',
        'b2t_battery_min_temp': 'battery_min_temp',
        'b2t_battery_max_temp': 'battery_max_temp',
        'coolant_in_temp': 'coolant_in',
        'coolant_out_temp': 'coolant_out',
        'ignitionstatus': 'ignition_status',
        'vehiclereadycondition': 'vehicle_ready',
        'gun_connection_status': 'gun_connected'
    })
    
    # Add a column for the type of transition
    transition_df['transition_type'] = 'both'
    transition_df.loc[transition_df['cmd_changed'] & ~transition_df['mode_changed'], 'transition_type'] = 'cmd_only'
    transition_df.loc[~transition_df['cmd_changed'] & transition_df['mode_changed'], 'transition_type'] = 'mode_only'
    
    # Drop temporary columns
    transition_df = transition_df.drop(columns=['cmd_changed', 'mode_changed'])
    
    # Calculate temperature differential
    transition_df['coolant_diff'] = transition_df['coolant_in'] - transition_df['coolant_out']
    
    logger.info(f"Identified {len(transition_df)} transitions")
    return transition_df

In [7]:
def analyze_charging_events(start_date, end_date, vehicle_ids):
    """
    Analyze transition data for all charging events identified by gun_connection_status.
    
    Args:
        start_date: Start date in 'YYYY-MM-DD' format
        end_date: End date in 'YYYY-MM-DD' format
        vehicle_ids: List of vehicle IDs to analyze
        
    Returns:
        DataFrame with transition data for all charging events
    """
    # Get all battery data for the specified date range and vehicles
    battery_data = get_all_battery_data(start_date, end_date, vehicle_ids)
    
    # Skip if no data
    if battery_data.empty:
        logger.info("No battery data found for the specified date range and vehicles")
        return pd.DataFrame()
    
    # Preprocess the data (including imputation for gun_connection_status)
    clean_data = preprocess_data(battery_data)
    
    # Identify charging events based on gun_connection_status
    charging_events = identify_charging_events(clean_data)
    
    # Skip if no charging events identified
    if charging_events.empty:
        logger.info("No charging events identified")
        return pd.DataFrame()
    
    # Assign battery data to charging events
    battery_with_events = assign_events_to_battery_data(clean_data, charging_events)
    
    # Skip if no data assigned to events
    if battery_with_events.empty:
        logger.info("No battery data assigned to charging events")
        return pd.DataFrame()
    
    # Get transition data
    transition_data = get_transition_data(battery_with_events)
    
    return transition_data

In [8]:
# Set date range and vehicle IDs
start_date = '2025-09-22'  # Format: 'YYYY-MM-DD'
end_date = '2025-09-25'    # Format: 'YYYY-MM-DD'
# vehicle_ids = [3]    # List of vehicle IDs

# Run the analysis
transition_df = analyze_charging_events(start_date, end_date, vehicle_ids)

2025-10-24 10:05:20 - INFO - Fetching all battery data from 2025-09-22 to 2025-09-25 for vehicles [19]
2025-10-24 10:05:50 - INFO - Retrieved 32568 battery data records
2025-10-24 10:05:51 - INFO - Data after cleaning: (32568, 21) (removed 0 rows)
2025-10-24 10:05:51 - INFO - Gun connection status distribution: {0: 23033, 1: 9535}
2025-10-24 10:05:51 - INFO - Identifying charging events based on gun_connection_status
2025-10-24 10:05:51 - INFO - Identified 15 charging events
2025-10-24 10:05:51 - INFO - Assigning battery data to charging events
/tmp/ipykernel_2479/4056704630.py:45: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '2025-09-22 05:54:04.182000+05:30' has dtype incompatible with datetime64[ns], please explicitly cast to a compatible dtype first.
  result_df.loc[mask & (result_df['id'] == vehicle_id), 'event_start_time'] = pd.to_datetime(event['start_time'])
/tmp/ipykernel_2479/4056704630.py:46: 

In [12]:
transition_df[transition_df.id == 19].head(50)

,id,timestamp,prev_cmd,current_cmd,prev_mode,current_mode,battery_min_temp,battery_max_temp,coolant_in,coolant_out,ignition_status,vehicle_ready,gun_connected,event_id,event_start_time,event_end_time,transition_type,coolant_diff
238,19,2025-09-22 05:54:04.182000+05:30,Unknown,Charging_Cooling,Unknown,Off,35.0,40.0,23.0,15.0,1.0,0.0,1,0,2025-09-22 05:54:04.182000+05:30,2025-09-22 06:31:00.882000+05:30,both,8.0
296,19,2025-09-22 06:31:00.882000+05:30,Charging_Cooling,Fast_Discharge_Cooling,Off,Off,34.0,39.0,26.0,23.0,1.0,0.0,0,0,2025-09-22 05:54:04.182000+05:30,2025-09-22 06:31:00.882000+05:30,cmd_only,3.0
2471,19,2025-09-22 13:42:20.316000+05:30,Fast_Discharge_Cooling,Charging_Cooling,Off,Off,34.0,37.0,11.0,10.0,NaN,NaN,1,1,2025-09-22 13:42:20.316000+05:30,2025-09-22 14:00:02.476000+05:30,cmd_only,1.0
2476,19,2025-09-22 13:42:25.576000+05:30,Charging_Cooling,Charging_Cooling,Off,Charging_Cooling,34.0,37.0,11.0,11.0,NaN,NaN,1,1,2025-09-22 13:42:20.316000+05:30,2025-09-22 14:00:02.476000+05:30,mode_only,0.0
2641,19,2025-09-22 13:59:00.756000+05:30,Charging_Cooling,Off,Charging_Cooling,Off,35.0,38.0,17.0,14.0,0.0,NaN,1,1,2025-09-22 13:42:20.316000+05:30,2025-09-22 14:00:02.476000+05:30,both,3.0
2642,19,2025-09-22 13:59:01.796000+05:30,Off,Fast_Discharge_Cooling,Off,Off,35.0,38.0,17.0,14.0,0.0,NaN,1,1,2025-09-22 13:42:20.316000+05:30,2025-09-22 14:00:02.476000+05:30,cmd_only,3.0
2646,19,2025-09-22 13:59:06.216000+05:30,Fast_Discharge_Cooling,Fast_Discharge_Cooling,Off,Fast_Discharge_Cooling,35.0,38.0,17.0,14.0,0.0,NaN,1,1,2025-09-22 13:42:20.316000+05:30,2025-09-22 14:00:02.476000+05:30,mode_only,3.0
4614,19,2025-09-22 20:35:00.797000+05:30,Fast_Discharge_Cooling,Charging_Cooling,Fast_Discharge_Cooling,Off,34.0,40.0,13.0,29.0,0.0,NaN,1,2,2025-09-22 20:35:00.797000+05:30,2025-09-22 21:07:00.928000+05:30,both,-16.0
4615,19,2025-09-22 20:35:01.958000+05:30,Charging_Cooling,Charging_Cooling,Off,Charging_Cooling,35.0,40.0,14.0,29.0,0.0,NaN,1,2,2025-09-22 20:35:00.797000+05:30,2025-09-22 21:07:00.928000+05:30,mode_only,-15.0
4946,19,2025-09-22 21:05:00.337000+05:30,Charging_Cooling,Off,Charging_Cooling,Off,35.0,41.0,21.0,29.0,0.0,NaN,1,2,2025-09-22 20:35:00.797000+05:30,2025-09-22 21:07:00.928000+05:30,both,-8.0
